In [1]:
using Serialization, Distributions, AffineInvariantMCMC, StableRNGs 
include("ode_problem.jl")
include("target_probability.jl")
include("define_likelihood_data.jl");

### Name Run

run: 295  
test data: pSHC  
augmented: no 
weighed by AF3 confidence: N/A   
prior offset: +/- 2   
binding prior offset: 0  
nwalkers = 500  
a = 1.1

In [2]:
run = "295"

"205"

Define Prior Distributions

In [3]:
#generate bounds for prior
bound_offset = [2,2]
binding_bound_offset = [0,0]
prior_bounds = return_prior_bounds_empirical(bound_offset, binding_bound_offset) #for target probability
prior_distributions = [Uniform(prior_bounds[i]...) for i in 1:length(prior_bounds)];

Define Test/Train Data

In [4]:
#first input we need are data points and each point's associated standard deviation to evaluate the likelihood
data_points = deserialize("outputs/000_training_data_w_pshc_test.dict")["response"]
std_dev = deserialize("outputs/000_training_data_w_pshc_test.dict")["average_error"]

#load different EGF ligand dosages needed to simulate all training data
ligand_dosages = return_ligand_dose_order_for_likelihood_w_pshc_test()

#provide function for indexing likelihood data - that is, non test data
training_data_indices = likelihood_w_pshc_test;

Define ODEProblems for different intial conditions

In [5]:
#next input we need is an ode problem
odeproblems = []
for egf in ligand_dosages
    odesys, u0, tspan, p = return_ode_problem_default_inputs(egf)
    prob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p) #note p will be overwritten in first run of mcmc algorithm
    push!(odeproblems, prob)
end

Define ODE Solver Inputs

In [6]:
#and ode solver inputs, which are consistent for all simulations
odesolver_inputs = return_ode_problem_solver_default_inputs();

Define Type of ODESolution

In [7]:
#get type of ODEProblem solution
pred = DifferentialEquations.solve(odeproblems[1], odesolver_inputs["solver"], abstol=odesolver_inputs["abstol"], reltol=odesolver_inputs["reltol"], saveat=odesolver_inputs["saveat"])
ode_sol_type = typeof(pred);

Define Augmented Likelihood Term

In [8]:
#none needed for baseline data set

Define Target Distribution for Sampling

In [8]:
target_probability = parameters -> logprob_baseline(parameters, prior_distributions, data_points, std_dev, training_data_indices, ligand_dosages, odeproblems, odesolver_inputs, ode_sol_type)

#51 (generic function with 1 method)

Affine Invariant Sampler Hyperparameters

In [9]:
#affine invariant sampler hyperparameters
numdims = 50
numwalkers = 500
thinning = 1
numsamples_perwalker = 100
burnin = 200
n_subchains = 10
a_burnin = 4
a = 1.1
rng = StableRNG(28475);

Run Affine Invariant Sampler from 0

In [11]:
x0 = Matrix{Float64}(undef,numdims,numwalkers)
for i in 1:numwalkers
    x0[:,i] = [rand(rng, prior_distributions[j]) for j in 1:numdims]
end

chain0, llhoodvals0 = AffineInvariantMCMC.sample(target_probability, numwalkers, x0, burnin, thinning, a_burnin, rng=rng)
serialize("outputs/$(run)_posterior_samples_subchain_0.jls", chain0)
serialize("outputs/$(run)_posterior_samples_likelihood_subchain_0.jls", llhoodvals0)

#release from RAM
chain0 = nothing
llhoodvals0 = nothing

for i in 1:n_subchains
    chain = deserialize("outputs/$(run)_posterior_samples_subchain_$(i-1).jls")
    chain, llhoodvals = AffineInvariantMCMC.sample(target_probability, numwalkers, chain[:, :, end], numsamples_perwalker, thinning, a, rng=rng)
    serialize("outputs/$(run)_posterior_samples_subchain_$(i).jls", chain)
    serialize("outputs/$(run)_posterior_samples_likelihood_subchain_$(i).jls", llhoodvals)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:06:13
Progress: 100%|█████████████████████████████████████████| Time: 0:07:10
Progress: 100%|█████████████████████████████████████████| Time: 0:06:55
Progress: 100%|█████████████████████████████████████████| Time: 0:06:52
Progress: 100%|█████████████████████████████████████████| Time: 0:06:53
Progress: 100%|█████████████████████████████████████████| Time: 0:06:52
Progress: 100%|█████████████████████████████████████████| Time: 0:06:53
Progress: 100%|█████████████████████████████████████████| Time: 0:06:54
Progress: 100%|█████████████████████████████████████████| Time: 0:06:55
Progress: 100%|█████████████████████████████████████████| Time: 0:06:54
Progress: 100%|█████████████████████████████████████████| Time: 0:06:54


Run Affine Invariate Sampler from N Subchain

In [10]:
#start = 51
#stop = 70
#for i in start:stop
    #chain = deserialize("outputs/$(run)_posterior_samples_subchain_$(i-1).jls")
    #chain, llhoodvals = AffineInvariantMCMC.sample(target_probability, numwalkers, chain[:, :, end], numsamples_perwalker, thinning, a, rng=rng)
    #serialize("outputs/$(run)_posterior_samples_subchain_$(i).jls", chain)
    #serialize("outputs/$(run)_posterior_samples_likelihood_subchain_$(i).jls", llhoodvals)
#end

┌ Warning: At t=5.194071257140165, dt was forced below floating point epsilon 8.881784197001252e-16, and step error estimate = 1.3213845952340433. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/Zrnkf/src/integrator_interface.jl:623
Progress:   1%|▎                                        |  ETA: 0:36:28┌ Warning: At t=31.954528048537842, dt was forced below floating point epsilon 3.552713678800501e-15, and step error estimate = 1.321384597748104. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/Zrnkf/src/integrator_interface.jl:623
Progress:   1%|▌                                        |  ETA: 0:34:49┌ Warning: At t=1.410413780813532, dt was forced below floatin